# ReactJS Notes for Building a Unit Converter Application

## Table of Contents

1. [Key Concepts](#key-concepts)
   - [State Management in React](#1-state-management-in-react)
   - [UI Rendering in React](#2-ui-rendering-in-react)
   - [Removing Repetition and Logic Reuse](#3-removing-repetition-and-logic-reuse)
2. [Original Code with Repetition](#original-code-with-repetition)
3. [Refactored Code with Reusable Component](#refactored-code-with-reusable-component)
   - [Logic Behind Reusable Component](#logic-behind-reusable-component)
4. [Understanding the Conversion Logic](#4-understanding-the-conversion-logic)
5. [Component Reusability and Scalability](#5-component-reusability-and-scalability)


## Key Concepts

### 1. State Management in React

- **State**: In React, state refers to the data or properties that control the behavior and rendering of components. Each component can have its own state, which can change over time. When state changes, React automatically re-renders the component to reflect those changes.
  
  Example of using state in React:
  
  ```jsx
  const [value, setValue] = React.useState(0);
  ```
  Here, `value` is the `state variable`, and `setValue` is the function used to `update the state`. `React.useState(0)` `initializes the state with a value of 0.`

- **State Updating:** To update the state, you use the setter function (`setValue`). React schedules a re-render of the component when the state updates.

    ```jsx
    const onChange = (event) => setValue(event.target.value);
    ```

### 2. UI Rendering in React
* **Rendering:** Rendering in React refers to the process of displaying a user interface based on the current state and props. Whenever the state or props change, React re-renders the component to reflect the updated state.

* **Dynamic Rendering:** React allows for dynamic UI updates without requiring a full page reload. This makes React applications fast and responsive.

### 3. Removing Repetition and Logic Reuse
* **Problem of Repetition:** Initially, the code for the unit converter application had a lot of repetition. Each conversion type (time, distance, area) had its own separate component, with almost identical code structures but different conversion factors. This led to redundancy and made the code harder to maintain.

    **Original Code with Repetition:**
    ```jsx
    //converter #1
    function TimeConverter() {
        const [time, setTime] = React.useState(0);
        const [selectedUnit, setSelected] = React.useState("seconds");
        const onChange = (event) => setTime(event.target.value);
        const reset = () => setTime(0);
        const onSelected = (unit) => {
            reset();
            setSelected(unit);
        };

        const timeConversionFactors = {
            seconds: { seconds: 1, minutes: 1 / 60, hours: 1 / 3600, days: 1 / 86400, years: 1 / (86400 * 365.25) },
            minutes: { seconds: 60, minutes: 1, hours: 1 / 60, days: 1 / 1440, years: 1 / (1440 * 365.25) },
            // ... other units
        };
        
        const convertTime = (targetUnit) => time * timeConversionFactors[selectedUnit][targetUnit];

        return (
            <div className="container">
                <h2>Time Converter</h2>
                {Object.keys(timeConversionFactors).map((unit) => (
                    <div className="input-group" key={unit}>
                        <label className="label" onClick={() => onSelected(unit)}>
                            {unit.charAt(0).toUpperCase() + unit.slice(1)}
                        </label>
                        <input
                            className="input"
                            value={selectedUnit === unit ? time : convertTime(unit)}
                            id={unit}
                            placeholder={`Enter ${unit}`}
                            type="number"
                            onChange={onChange}
                            disabled={selectedUnit !== unit}
                        />
                    </div>
                ))}
                <button className="button" onClick={reset}>Reset</button>
            </div>
        );
    }
    //converter #2
    function DistanceConverter() {
        const [distance, setDistance] = React.useState(0);
        const [selectedUnit, setSelected] = React.useState("miles");
        const onChange = (event) => setDistance(event.target.value);
        const reset = () => setDistance(0);
        const onSelected = (unit) => {
            reset();
            setSelected(unit);
        };

        const distanceConversionFactors = {
            miles: { miles: 1, kilometers: 1.60934, meters: 1609.34, yards: 1760, feet: 5280, inches: 63360 },
            kilometers: { miles: 0.621371, kilometers: 1, meters: 1000, yards: 1093.61, feet: 3280.84, inches: 39370.1 },
            // ... other units
        };
        
        const convertDistance = (targetUnit) => distance * distanceConversionFactors[selectedUnit][targetUnit];

        return (
            <div className="container">
                <h2>Distance Converter</h2>
                {Object.keys(distanceConversionFactors).map((unit) => (
                    <div className="input-group" key={unit}>
                        <label className="label" onClick={() => onSelected(unit)}>
                            {unit.charAt(0).toUpperCase() + unit.slice(1)}
                        </label>
                        <input
                            className="input"
                            value={selectedUnit === unit ? distance : convertDistance(unit)}
                            id={unit}
                            placeholder={`Enter ${unit}`}
                            type="number"
                            onChange={onChange}
                            disabled={selectedUnit !== unit}
                        />
                    </div>
                ))}
                <button className="button" onClick={reset}>Reset</button>
            </div>
        );
    }

    // ... Similar code for AreaConverter
    ```
Each converter component above has almost identical logic and structure, except for the different conversion factors and titles.

* **Solution:** Reusable Components: To reduce repetition, we refactored the code to create a reusable `UnitConverter` component. This component accepts `conversionFactors, symbols, and title as props`, allowing it to handle different types of conversions dynamically.

    **Refactored Code with Reusable Component:**

    ```jsx
    function UnitConverter({ conversionFactors, symbols, title }) {
        const [value, setValue] = React.useState(0);
        const [selectedUnit, setSelectedUnit] = React.useState(Object.keys(conversionFactors)[0]);
        
        const onChange = (event) => setValue(event.target.value);
        const reset = () => setValue(0);
        const onSelected = (unit) => {
            reset();
            setSelectedUnit(unit);
        };

        const convertValue = (targetUnit) => value * conversionFactors[selectedUnit][targetUnit];

        return (
            <div className="container">
                <h2>{title}</h2>
                {Object.keys(conversionFactors).map((unit) => (
                    <div className="input-group" key={unit}>
                        <label className="label" onClick={() => onSelected(unit)}>
                            {unit.charAt(0).toUpperCase() + unit.slice(1)} ({symbols[unit]})
                        </label>
                        <input
                            className="input"
                            value={selectedUnit === unit ? value : convertValue(unit)}
                            id={unit}
                            placeholder={`Enter ${unit}`}
                            type="number"
                            onChange={onChange}
                            disabled={selectedUnit !== unit}
                        />
                    </div>
                ))}
                <button className="button" onClick={reset}>Reset</button>
            </div>
        );
    }
    ```
* **Logic Behind Reusable Component:**

    * **Props for Flexibility:** The `UnitConverter` component takes `conversionFactors, symbols, and title` as props. This allows the component to be reused for different types of conversions (time, distance, area) without changing the internal logic.
    * **Dynamic Rendering with `map()`:** Instead of hardcoding each unit conversion input, we use `Object.keys(conversionFactors).map((unit) => ...)` to dynamically generate input fields for each unit. This approach keeps the code DRY (Don't Repeat Yourself) and makes it easy to update or add new units.
    * **Event Handling:** We handle user interactions such as changing input values, switching units, and resetting the state through event handlers (`onChange, onSelected, reset`), which update the component's state and trigger re-renders.

### 4. Understanding the Conversion Logic
* **Conversion Factors:** The conversion factors are stored in an object where each key is a unit, and the corresponding value is another object that defines conversion rates to other units.
    ```jsx
    const timeConversionFactors = {
        seconds: { seconds: 1, minutes: 1 / 60, hours: 1 / 3600, days: 1 / 86400, years: 1 / (86400 * 365.25) },
        minutes: { seconds: 60, minutes: 1, hours: 1 / 60, days: 1 / 1440, years: 1 / (1440 * 365.25) },
        // ... other units
    };
    ```
* **Dynamic Conversion Calculation:** The `convertValue` function dynamically calculates the conversion based on the selected unit and target unit using the `conversionFactors` object.

    ```jsx
    const convertValue = (targetUnit) => value * conversionFactors[selectedUnit][targetUnit];
    ```
### 5. Component Reusability and Scalability
* **Reusable UI Components:** The UnitConverter component can be easily reused for any unit conversion by simply passing different conversionFactors and symbols. This approach minimizes code duplication and makes the application more maintainable and scalable.

* **Scalability:** To add new conversion types (e.g., currency, weight), you simply create new conversion factor objects and pass them to the `UnitConverter` component. This modularity allows the application to scale without significant code refactoring.


